# Converting Rosanne's magnification files to magnified light curves and with the PLASTICC/SNANA format

### USER

In [13]:
# Dir of magnification files (those provided by Rosanne Di Stefano)
DirLensFiles = "/Users/arturo/Documents/Research/LSST/2018_07_10_a/1_Magnification/"

# Dir of simulated LSST star catalog
DirStarCatalog = "/Users/arturo/Documents/Research/LSST/2018_07_10_a/"

# Dir save output
DirSaveOutput = "/Users/arturo/Documents/Research/LSST/2018_07_10_a/"

# -------------------------------
# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand
NotebookName = 'Create_magnified_LCs.ipynb'

### Automatic

In [23]:
import numpy as np
import pandas as pd
import os # To use command line like instructions
import glob # To read the files in my directory
from matplotlib import pyplot as plt

5+6

11

#### Get the name of this ipython notebook
To print it in the output text files as reference.

In [5]:
%%javascript
var kernel = IPython.notebook.kernel;
var thename = window.document.getElementById("notebook_name").innerHTML;
var command = "NotebookName = " + "'"+thename+".ipynb"+"'";
kernel.execute(command);

<IPython.core.display.Javascript object>

In [6]:
print(NotebookName)

# Given that javascript doesn't work in JupyterLab, I have to set the 
# name of the notebook by hand

Create_magnified_LCs.ipynb


In [7]:
# Get the current date and time
import datetime 

# Read the time and date now
now = datetime.datetime.now()

In [8]:
# Function to convert from magnification to apparent magnitude

def Magnification2mag(Magf, mo):
    mag_int = mo - 2.5*np.log10(Magf)
    return mag_int

print '# Test:'
print "#", Magnification2mag(2.14118954, 19.46)

# Test
# 18.6333622172
# 18.633362217176515

# Test:
# 18.6333622172


In [14]:
# Read the LSST star catalogue file with the magnitudes in ugrizY bands

starsdata = np.genfromtxt(DirStarCatalog+'2_lsst_star_table_sample.txt',
                dtype=[float,float,float,float,float,
                       float,float,float]) 

print "# %s stars data found in the file"%len(starsdata)
# 35 stars data found in the file

# 35 stars data found in the file


In [ ]:
starsdata['f2']

In [ ]:
"""
array([ 16.21739,  18.41886,  27.47009,  19.72312,  30.9791 ,  26.53289,
        31.61692,  18.70061,  18.97976,  19.42187,  24.40802,  19.36606,
        17.46127,  29.19847,  30.27772,  23.00819,  24.09528,  21.57186,
        14.83046,  27.84922,  27.55117,  23.27528,  24.47566,  23.22603,
        27.1417 ,  27.52953,  25.5453 ,  27.31056,  26.48858,  24.22798,
        25.92916,  27.09815,  23.15348,  25.65296,  27.26104])
"""

In [19]:
import os # To use command line like instructions
import glob # To read the files in my directory

# Read the metadata file:
lens_metadata = np.genfromtxt(DirLensFiles+'tau.dist_information.txt',
                             dtype=[int,int,float,float,float,float])

#------------------------------

#- Force the creation of the directory to save the plots.
#- "If the subdirectory does not exist then create it"
import os # To use command line like instructions
if not os.path.exists(DirSaveOutput+"2_plots"): os.makedirs(DirSaveOutput+"2_plots")

In [20]:
len(lens_metadata)

16000

In [21]:
lens_metadata[0]

(1, 197,  48.48,  1.602, -0.1665,  1.112)

In [ ]:
# Reading the parameter values used to create every "fort" file
# these values are just to keep in the record and write down on 
# the Plasticc/SNANA output file.

# ParsModel=np.genfromtxt(DirData+'1_Magnification/parameters_Binary_lens.txt')
# print "# %s value sets were found in this file."%len(ParsModel)

In [69]:
# Number label of the light curve:
lc_num = 1
# Number of observations in that light curve:
lc_num_obs = 197


In [85]:
initial_row = 0

for j in range(4):
    
    # Number of observations in that light curve:
    lc_num_obs = lens_metadata[j][1]
    
    number_rows = lc_num_obs
    
    lens_data = pd.read_table(DirLensFiles+'tau.dist_LCs.dat', 
                      skiprows=initial_row, nrows=number_rows, sep='\s+')
    print initial_row, number_rows
    print lens_data.values[0]
    
    # Increase the value of "initial_row"
    initial_row = initial_row + number_rows

    

0 197
[ 1.      -0.16646  1.     ]
197 30
[ 1.       0.16514  1.     ]
227 84
[ 1.       0.11738  1.     ]
311 84
[ 1.       -0.005246  1.      ]


In [79]:
lens_data = pd.read_table(DirLensFiles+'tau.dist_LCs.dat', 
                      skiprows=initial_row, nrows=number_rows, sep='\s+')

In [80]:
test1

,197,253.14,1.000
0,1,0.16514,1.000
1,2,12.68600,1.001
2,3,13.95700,1.002
3,4,14.65500,1.003
4,5,15.14400,1.004
5,6,15.51900,1.005
6,7,15.83200,1.006
7,8,16.10300,1.007
8,9,16.34200,1.008
9,10,16.57100,1.009


In [68]:
test1.values

array([[  1.00000000e+00,  -1.66460000e-01,   1.00000000e+00],
       [  2.00000000e+00,   2.36060000e+02,   1.00100000e+00],
       [  3.00000000e+00,   2.37200000e+02,   1.00200000e+00]])

In [62]:
test1.values[0]

array([ 1.     , -0.16646,  1.     ])

In [63]:
test1.values[0][0]

1.0

In [64]:
test1.values[0][0][0]

IndexError: invalid index to scalar variable.

### Main loop. v 1.3

In [ ]:
# Difference in u magnitude as a criterium to write down 
# that row in the text file. This is helpful to avoid writting a lot of
# rows with exactly the same values of the magnitudes in ugrizY bands.
diffmag_fix = 0.001

# Maximum percentage of time of a LC with magnitude larger
# than 16 mags
Porcentage_maxTime = 18.0 # percentage

# Magnitude threshold
maxMag = 16.0 # mag

#------------------------------------------

textfile_1 = open(DirSaveOutput+'2_Lensing_lc_PLASTICC_format_.txt', 'w')

now = datetime.datetime.now() # Read the time and date right now
text_timenow = now.strftime("%Y-%m-%d (yy-mm-dd).")
text_Date   = 'COMMENT: File created by Arturo Avelino on %s \n'%text_timenow
text_line = '#'+'-'*50 + '\n'
    
textfile_1.write("SURVEY: LSST \n")
textfile_1.write("FILTERS: ugrizY \n")
textfile_1.write("MODEL: Microlensing by Rosanne Di Stefano \n")
textfile_1.write("MODEL_PARNAMES: apeak,m1,m2,alpha,d_l,v,i_p,omega,clock,\
beta,nom,porb,tau \n")
textfile_1.write(text_Date)
# textfile_1.write()

countEvent = 0

print "# Number of useful rows written in the text file for given star,"
print "# it is, when there is a change in magnitude at least > %s mag."%diffmag_fix

# Reset variables.
initial_row = 0

# old. for j in range(len(list_forts)): # Loop over the stars
for j in range(10): # Loop over the light curves (i.e., stars) # tmp  
    
    ra  = starsdata['f0'][j]
    dec = starsdata['f1'][j]
    
    # Apparent magnitude star in different bands:
    magStar_u = starsdata['f2'][j]
    magStar_g = starsdata['f3'][j]
    magStar_r = starsdata['f4'][j]
    magStar_i = starsdata['f5'][j]
    magStar_z = starsdata['f6'][j]
    magStar_y = starsdata['f7'][j]
    
    # Number label of the current light curve:
    lc_num = lens_metadata[j][0]
    # Number of observations in the current light curve:
    lc_num_obs = lens_metadata[j][1]
    
    #------------------------------------------
    # Uploading the magnification table.
    # It is a very large file so I use pandas to read the datatable
    # by portions.
    lens_data = pd.read_table(DirLensFiles+'tau.dist_LCs.dat', 
                      skiprows=initial_row, nrows=lc_num_obs, sep='\s+')
    
    print initial_row, lc_num_obs
    print lens_data.values[0]
    
    # Increase the value of "initial_row" for the next 
    # iteration in the main loop.
    initial_row = initial_row + lc_num_obs
    
    ##########################################################
    
    # Loop over the magnification file for a given star
    # I use this loop just to count the number of rows and to
    # discard the magnified stars with magnitudes larger than
    # 16 mags for more than a given porcentage of time.
    
    # Reset the counter for the number of rows for this event
    # based on the change in magnitude in u band:
    countNROW = 0
    count_AboveMaxMag = 0
    
    # oldx. for ii in range(len(lensdata)):
    for ii in range(lc_num_obs):

        Magnific = lens_data[ii][3]
        mag_u = Magnification2mag(Magnific, magStar_u)
        
        # Compute the difference between the previous magnitude 
        # and the current value:
        if ii > 0:
            Magnific_prev = lensdata[int(ii-1)][3]
            mag_u_prev = Magnification2mag(Magnific_prev, magStar_u)
            diffmag = abs(round(mag_u,3) - round(mag_u_prev,3))
        else: diffmag = 1
        
        if diffmag >= diffmag_fix: 
            countNROW += 1
            if mag_u < maxMag:
                count_AboveMaxMag += 1
                
    # Determine the porcentage of MJD with mag larger than 
    # the magnitude threshold:
    porcentage_int1 = (float(count_AboveMaxMag)*100.)/(float(countNROW+3))
    
    #########################################

    # Consider the LCs that have less than the maximum 
    # porcentage of time of a LC with magnitude larger
    # than the threshold magnitude only.
    
    if porcentage_int1 < Porcentage_maxTime:
        
        countEvent = countEvent + 1

        textfile_1.write(" \n")
        textfile_1.write(text_line)
        textfile_1.write(" \n")
        textfile_1.write("START_EVENT: %s \n"%countEvent)
        
        # Write the metadata info for this star.
        textfile_1.write("NROW: %s  RA: %s   DEC: %s  \n"%(
            (countNROW+3), ra, dec))
    
        #-----------------------------------------

        # Write the parameters of the specific event:

        textfile_1.write("PARVAL: ")
        for k in range(len(ParsModel[j])):
            textfile_1.write("%.3f  "%ParsModel[j][k])
        textfile_1.write(" \n")

        #---------------------------------

        time_first = int(lensdata[0][2]-1) # initial time

        # Magnitudes of the template
        textfile_1.write("T: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(
            time_first, magStar_u, magStar_g, magStar_r, magStar_i,
            magStar_z, magStar_y))
        
        # Write the first event: it is equal in mags to template
        textfile_1.write("S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(
            time_first, magStar_u, magStar_g, magStar_r, magStar_i,
            magStar_z, magStar_y))

        #---------------------------------------------------
        # Loop over the magnification file for a given star

        # Creating the plot 
        plt.figure()

        time_list = []
        mag_u_list = []
        mag_g_list = []

        time_list  += [time_first]
        mag_u_list += [magStar_u]
        mag_g_list += [magStar_g]

        countNROW_2 = 0
        for ii in range(len(lensdata)):

            time_int = lensdata[ii][2]
            Magnific = lensdata[ii][3]

            mag_u = Magnification2mag(Magnific, magStar_u)
            mag_g = Magnification2mag(Magnific, magStar_g)
            mag_r = Magnification2mag(Magnific, magStar_r)
            mag_i = Magnification2mag(Magnific, magStar_i)
            mag_z = Magnification2mag(Magnific, magStar_z)
            mag_y = Magnification2mag(Magnific, magStar_y)

            # Compute the difference between the previous magnitude 
            # and the current value, and print only the times
            # when the difference is larger than "diffmag_fix"
            if ii > 0:
                Magnific_prev = lensdata[int(ii-1)][3]
                mag_u_prev = Magnification2mag(Magnific_prev, magStar_u)
                diffmag = abs(round(mag_u,3) - round(mag_u_prev,3))
            else: diffmag = 1

            if diffmag >= diffmag_fix:

                text_05 = 'S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  %.3f \n'%(
                    time_int, mag_u, mag_g, mag_r, mag_i, mag_z, mag_y)
                textfile_1.write(text_05)

                countNROW_2 = countNROW_2 + 1

                time_list  += [time_int]
                mag_u_list += [mag_u] 
                mag_g_list += [mag_g]  
                
                # A hack to define the last MJD. I will use this to 
                # write down the last event where the MJD will the the
                # last used time + 1 day, and the magnitudes will
                # be the same than the template.
                time_int2 = time_int

        # Write the last event: it is equal in mags to template
        textfile_1.write("S: %9.4f  %.3f  %.3f  %.3f  %.3f  %.3f  \
%.3f \n"%(
            (time_int2+1), magStar_u, magStar_g, magStar_r, magStar_i,
            magStar_z, magStar_y))        
        
        
        #------------- PLOT THE LCs -------------------
        
        mag_u_np = np.array(mag_u_list)
        
        # the number is to create an offset:
        # mag_g_np = np.array(mag_g_list) + 2 

        # Plot with points and lines (recommended)
        plt.plot(time_list, mag_u_list, lw=0.5, marker=".", ms=2) 
        
        # just the line:
        # plt.plot(time_list, mag_u_list, lw=2, ls = "-", color = 'blue') 
        # plt.plot(time_list, mag_g_list, lw=2, ls = "-", color = 'green') 

        plt.xlim(min(time_list)-100, max(time_list)+50)
        plt.ylim(max(mag_u_list)+0.1, min(mag_u_list)-0.1)

        plt.xlabel('time (days)')
        plt.ylabel('apparent magnitude')
        plt.title('u-band light curves of a lensed LSST star')

        plt.savefig(DirSaveOutput+'2_plots/plot_%s.png'%countEvent, dpi=120)
        plt.close()
        textfile_1.write("END_EVENT: %s \n"%countEvent)
        print "%s: %s, %s countNROW"%(j, countNROW,countNROW_2)

textfile_1.close()